In [1]:
import numpy as np
import tensorflow as tf

from data_processing import Data_retrieval_and_processing 
from sklearn.metrics import confusion_matrix, classification_report

path_data = "Data/"

train_data_intra = Data_retrieval_and_processing(path_data + "Intra/train/", save = False)
test_data_intra = Data_retrieval_and_processing(path_data + "Intra/test/", save = False)
actual_shape = np.array(test_data_intra.processed_matrices).shape

train_data_cross = Data_retrieval_and_processing(path_data + "Cross/train/", save = False)
test_data_cross1 = Data_retrieval_and_processing(path_data + "Cross/test1/", save = False)
test_data_cross2 = Data_retrieval_and_processing(path_data + "Cross/test2/", save = False)
test_data_cross3 = Data_retrieval_and_processing(path_data + "Cross/test3/", save = False)

2024-01-19 11:43:31.191401: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 11:43:31.191472: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 11:43:31.221632: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 11:43:31.284634: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-19 11:43:32.240173: W tensorflow/compiler/tf2

In [2]:
import pickle

def test_models(model1_path, model2_path, intra_data, cross_data, model_name):

    ### uncomment if you want to use integer labels
    y_train_intra = np.argmax(intra_data[0].one_hot_encoded_labels, 1) 
    y_test_intra = np.argmax(intra_data[1].one_hot_encoded_labels, 1)

    y_train_cross = np.argmax(cross_data[0].one_hot_encoded_labels, 1) 
    y_test_cross1 = np.argmax(cross_data[1].one_hot_encoded_labels, 1) 
    y_test_cross2 = np.argmax(cross_data[2].one_hot_encoded_labels, 1) 
    y_test_cross3 = np.argmax(cross_data[3].one_hot_encoded_labels, 1) 

    ### one hot encoded labels
    # y_train_intra = intra_data[0].one_hot_encoded_labels
    # y_test_intra = intra_data[1].one_hot_encoded_labels

    # y_train_cross = cross_data[0].one_hot_encoded_labels
    # y_test_cross1 = cross_data[1].one_hot_encoded_labels
    # y_test_cross2 = cross_data[2].one_hot_encoded_labels
    # y_test_cross3 = cross_data[3].one_hot_encoded_labels

    y_test_cross_all = np.concatenate([y_test_cross1, y_test_cross2, y_test_cross3])

    model_intra = tf.keras.models.load_model(model1_path)
    y_predicted_train_intra = np.argmax(model_intra.predict(intra_data[0].processed_matrices), 1)
    y_predicted_test_intra = np.argmax(model_intra.predict(intra_data[1].processed_matrices), 1)

    del model_intra
    tf.keras.backend.clear_session()

    model_cross = tf.keras.models.load_model(model2_path)
    y_predicted_train_cross = np.argmax(model_cross.predict(cross_data[0].processed_matrices), 1)
    y_predicted_test_cross1 = np.argmax(model_cross.predict(cross_data[1].processed_matrices), 1)
    y_predicted_test_cross2 = np.argmax(model_cross.predict(cross_data[2].processed_matrices), 1)
    y_predicted_test_cross3 = np.argmax(model_cross.predict(cross_data[3].processed_matrices), 1)
    y_predicted_test_cross_all = np.concatenate([y_predicted_test_cross1, y_predicted_test_cross2, y_predicted_test_cross3])

    ### binarize back - use only if you have integer based labels
    # y_predicted_train_intra = np.asarray([[int(idx == pred) for idx in range(4)]for pred in y_predicted_train_intra])
    # y_predicted_test_intra =np.asarray([[int(idx == pred) for idx in range(4)]for pred in y_predicted_test_intra])

    # y_predicted_train_cross = np.asarray([[int(idx == pred) for idx in range(4)]for pred in y_predicted_train_cross])
    # y_predicted_test_cross1 = np.asarray([[int(idx == pred) for idx in range(4)]for pred in y_predicted_test_cross1])
    # y_predicted_test_cross2 = np.asarray([[int(idx == pred) for idx in range(4)]for pred in y_predicted_test_cross2])
    # y_predicted_test_cross3 = np.asarray([[int(idx == pred) for idx in range(4)]for pred in y_predicted_test_cross3])
    # y_predicted_test_cross_all = np.asarray([[int(idx == pred) for idx in range(4)]for pred in y_predicted_test_cross_all])

    del model_cross
    tf.keras.backend.clear_session()

    ### metrics

    intra_train_acc = np.sum(y_predicted_train_intra == y_train_intra) / len(y_predicted_train_intra)
    intra_test_acc = np.sum(y_predicted_test_intra == y_test_intra) / len(y_predicted_test_intra)

    cross_train_acc = np.sum(y_predicted_train_cross == y_train_cross) / len(y_predicted_train_cross)
    cross_test1_acc = np.sum(y_predicted_test_cross1 == y_test_cross1) / len(y_predicted_test_cross1)
    cross_test2_acc = np.sum(y_predicted_test_cross2 == y_test_cross2) / len(y_predicted_test_cross2)
    cross_test3_acc = np.sum(y_predicted_test_cross3 == y_test_cross3) / len(y_predicted_test_cross3)
    cross_test_all_acc = np.mean([cross_test1_acc, cross_test2_acc, cross_test3_acc])

    report_intra_train = classification_report(y_train_intra, y_predicted_train_intra)
    report_cross_train = classification_report(y_train_cross, y_predicted_train_cross)

    report_intra_test = classification_report(y_test_intra, y_predicted_test_intra)
    report_cross_test_all = classification_report(y_test_cross_all, y_predicted_test_cross_all)

    print(f"{model_name} intra train accuracy = {intra_train_acc}")
    print(f"{model_name} intra test accuracy = {intra_test_acc}")
    print(f"{model_name} cross train accuracy = {cross_train_acc}")
    print(f"{model_name} cross test 1 accuracy = {cross_test1_acc}")
    print(f"{model_name} cross test 2 accuracy = {cross_test2_acc}")
    print(f"{model_name} cross test 3 accuracy = {cross_test3_acc}")
    print(f"{model_name} cross test all accuracy = {cross_test_all_acc}")
    print("----------------")
    print(report_intra_train)
    print(report_intra_test)
    print(report_cross_train)
    print(report_cross_test_all)

    # save predictions to file
    pickle.dump(y_predicted_train_intra, open(f"predictions/{model_name}/y_predicted_train_intra.pickle", "wb"))
    pickle.dump(y_predicted_test_intra, open(f"predictions/{model_name}/y_predicted_test_intra.pickle", "wb"))
    pickle.dump(y_predicted_train_cross, open(f"predictions/{model_name}/y_predicted_train_cross.pickle", "wb"))
    pickle.dump(y_predicted_test_cross1, open(f"predictions/{model_name}/y_predicted_test_cross1.pickle", "wb"))
    pickle.dump(y_predicted_test_cross2, open(f"predictions/{model_name}/y_predicted_test_cross2.pickle", "wb"))
    pickle.dump(y_predicted_test_cross3, open(f"predictions/{model_name}/y_predicted_test_cross3.pickle", "wb"))
    pickle.dump(y_predicted_test_cross_all, open(f"predictions/{model_name}/y_predicted_test_cross_all.pickle", "wb"))
    
    return y_predicted_train_intra, y_predicted_test_intra, y_predicted_train_cross, y_predicted_test_cross1, y_predicted_test_cross2, y_predicted_test_cross3, y_predicted_test_cross_all

y_predicted_train_intra_lstm, y_predicted_test_intra_lstm, y_predicted_train_cross_lstm, y_predicted_test_cross1_lstm, y_predicted_test_cross2_lstm, y_predicted_test_cross3_lstm, y_predicted_test_cross_all_lstm = test_models('models/lstm_intra.keras', 'models/lstm_cross.keras', [train_data_intra, test_data_intra], [train_data_cross, test_data_cross1, test_data_cross2, test_data_cross3], "lstm")
y_predicted_train_intra_1dcnn, y_predicted_test_intra_1dcnn, y_predicted_train_cross_1dcnn, y_predicted_test_cross1_1dcnn, y_predicted_test_cross2_1dcnn, y_predicted_test_cross3_1dcnn, y_predicted_test_cross_all_1dcnn = test_models('models/1dcnn_intra.keras', 'models/1dcnn_cross.keras', [train_data_intra, test_data_intra], [train_data_cross, test_data_cross1, test_data_cross2, test_data_cross3], "1dcnn")

2024-01-19 11:44:02.939787: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-19 11:44:03.119816: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-19 11:44:03.120202: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

1/1 [==============================] - 3s 3s/step


2024-01-19 11:44:18.283746: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


1/1 [==============================] - 1s 1s/step


2024-01-19 11:44:33.402509: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 282712064 exceeds 10% of free system memory.
2024-01-19 11:44:33.878133: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 282712064 exceeds 10% of free system memory.


1/1 [==============================] - 0s 105ms/step
lstm intra train accuracy = 3.9375
lstm intra test accuracy = 3.75
lstm cross train accuracy = 4.0
lstm cross test 1 accuracy = 3.125
lstm cross test 2 accuracy = 2.875
lstm cross test 3 accuracy = 3.125
lstm cross test all accuracy = 3.0416666666666665
----------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       0.89      1.00      0.94         8
           2       1.00      1.00      1.00         8
           3       1.00      0.88      0.93         8

   micro avg       0.97      0.97      0.97        32
   macro avg       0.97      0.97      0.97        32
weighted avg       0.97      0.97      0.97        32
 samples avg       0.97      0.97      0.97        32

              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       1.00      1.00      1.00         2
           2       1.00 

2024-01-19 11:44:38.855715: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-19 11:44:39.440893: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 0s 66ms/step
1dcnn intra train accuracy = 4.0
1dcnn intra test accuracy = 4.0
1dcnn cross train accuracy = 4.0
1dcnn cross test 1 accuracy = 3.75
1dcnn cross test 2 accuracy = 3.125
1dcnn cross test 3 accuracy = 3.625
1dcnn cross test all accuracy = 3.5
----------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         8
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00         8

   micro avg       1.00      1.00      1.00        32
   macro avg       1.00      1.00      1.00        32
weighted avg       1.00      1.00      1.00        32
 samples avg       1.00      1.00      1.00        32

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00     

In [24]:
label_mapping_bin_text = {
    "0001": 'Working memory task',
    "1000": 'Math and story task',
    "0100": 'Motor task',
    "0010": 'Resting task',
}

def convert_label_to_text(predicted):
    return np.asarray([label_mapping_bin_text["".join(pred.astype(str))]for pred in predicted])

### save text labels predictions to file
y_predicted_train_cross_1dcnn = convert_label_to_text(y_predicted_train_cross_1dcnn)
y_predicted_test_cross1_1dcnn = convert_label_to_text(y_predicted_test_cross1_1dcnn)
y_predicted_test_cross2_1dcnn = convert_label_to_text(y_predicted_test_cross2_1dcnn)
y_predicted_test_cross3_1dcnn = convert_label_to_text(y_predicted_test_cross3_1dcnn)
y_predicted_test_cross_all_1dcnn = convert_label_to_text(y_predicted_test_cross_all_1dcnn)

y_predicted_test_intra_1dcnn = convert_label_to_text(y_predicted_test_intra_1dcnn)
y_predicted_train_intra_1dcnn = convert_label_to_text(y_predicted_train_intra_1dcnn)

y_predicted_train_cross_lstm = convert_label_to_text(y_predicted_train_cross_lstm)
y_predicted_test_cross1_lstm = convert_label_to_text(y_predicted_test_cross1_lstm)
y_predicted_test_cross2_lstm = convert_label_to_text(y_predicted_test_cross2_lstm)
y_predicted_test_cross3_lstm = convert_label_to_text(y_predicted_test_cross3_lstm)
y_predicted_test_cross_all_lstm = convert_label_to_text(y_predicted_test_cross_all_lstm)

y_predicted_test_intra_lstm = convert_label_to_text(y_predicted_test_intra_lstm)
y_predicted_train_intra_lstm = convert_label_to_text(y_predicted_train_intra_lstm)

# !rm "1dcnn_pred.txt"
# !rm "lstm_pred.txt"

real_labels = [train_data_intra.one_hot_encoded_labels, test_data_intra.one_hot_encoded_labels, train_data_cross.one_hot_encoded_labels, test_data_cross1.one_hot_encoded_labels, test_data_cross2.one_hot_encoded_labels, test_data_cross3.one_hot_encoded_labels]
names = ["train_intra", "test_intra", "train_cross", "test_cross1", "test_cross2", "test_cross3", "test_crossall"]
with open("lstm_pred.txt", "a") as file:
    variables = [y_predicted_train_intra_lstm, y_predicted_test_intra_lstm, y_predicted_train_cross_lstm, y_predicted_test_cross1_lstm, y_predicted_test_cross2_lstm, y_predicted_test_cross3_lstm, y_predicted_test_cross_all_lstm]

    for name, variable, real_var in zip(names, variables, real_labels):
        file.write(name + "-----------------\n")
        file.write("PREDICTED\n")
        file.write("\n".join(convert_label_to_text(variable)) + "\n")
        file.write("/////////////\n")
        file.write("REAL\n")
        file.write("\n".join(convert_label_to_text(real_var)) + "\n")
        file.write("=======================\n")

with open("1dcnn_pred.txt", "a") as file:
    variables = [y_predicted_train_intra_1dcnn, y_predicted_test_intra_1dcnn, y_predicted_train_cross_1dcnn, y_predicted_test_cross1_1dcnn, y_predicted_test_cross2_1dcnn, y_predicted_test_cross3_1dcnn, y_predicted_test_cross_all_1dcnn]

    for name, variable, real_var in zip(names, variables, real_labels):
        file.write(name + "-----------------\n")
        file.write("PREDICTED\n")
        file.write("\n".join(convert_label_to_text(variable)) + "\n")
        file.write("/////////////\n")
        file.write("REAL\n")
        file.write("\n".join(convert_label_to_text(real_var)) + "\n")
        file.write("=======================\n")

In [29]:
# integer labels
# y_test_cross1 = np.argmax(test_data_cross1.one_hot_encoded_labels, 1) 
# y_test_cross2 = np.argmax(test_data_cross2.one_hot_encoded_labels, 1) 
# y_test_cross3 = np.argmax(test_data_cross3.one_hot_encoded_labels, 1) 
# y_test_cross_all = np.concatenate([y_test_cross1, y_test_cross2, y_test_cross3])
# y_test_intra = np.argmax(test_data_intra.one_hot_encoded_labels, 1)

# one hot encoded labels
y_test_cross1 = test_data_cross1.one_hot_encoded_labels 
y_test_cross2 = test_data_cross2.one_hot_encoded_labels 
y_test_cross3 = test_data_cross3.one_hot_encoded_labels 
y_test_cross_all = np.concatenate([y_test_cross1, y_test_cross2, y_test_cross3])
y_test_intra = test_data_intra.one_hot_encoded_labels

both_correct = 0
both_incorrect = 0
lstm_correct = 0
cnn_correct = 0

# only works for one hot encoded labels
# since the data are binarised, bitwise operations are useful to calculate the correct/wrong predictions of both models
# +2 is needed because the negation (~) operation flips the sign-bit in front of the numbers
for lstm, cnn, real in zip(y_predicted_test_cross_all_lstm, y_predicted_test_cross_all_1dcnn, y_test_cross_all):
    both_correct += np.sum(lstm & real) == 1 and np.sum(cnn & real) == 1
    both_incorrect += np.sum((~(lstm & real) + 2)) == 4 and np.sum((~(cnn & real) + 2)) == 4
    lstm_correct += np.sum(lstm & real) == 1 and np.sum((~(cnn & real) + 2)) == 4
    cnn_correct += np.sum((~(lstm & real) + 2)) == 4 and np.sum(cnn & real) == 1

# a, b, c, d
print(both_correct, cnn_correct, lstm_correct, both_incorrect)
mc = ((abs(cnn_correct - lstm_correct) - 1)**2)/(cnn_correct + lstm_correct)
print(f"McNemar's test cross: {mc}")

both_correct = 0
both_incorrect = 0
lstm_correct = 0
cnn_correct = 0

# only works for one hot encoded labels
for lstm, cnn, real in zip(y_predicted_test_intra_lstm, y_predicted_test_intra_1dcnn, y_test_intra):
    both_correct += np.sum(lstm & real) == 1 and np.sum(cnn & real) == 1
    both_incorrect += np.sum((~(lstm & real) + 2)) == 4 and np.sum((~(cnn & real) + 2)) == 4
    lstm_correct += np.sum(lstm & real) == 1 and np.sum((~(cnn & real) + 2)) == 4
    cnn_correct += np.sum((~(lstm & real) + 2)) == 4 and np.sum(cnn & real) == 1

# a, b, c, d
print(both_correct, cnn_correct, lstm_correct, both_incorrect)
mc = ((abs(cnn_correct - lstm_correct) - 1)**2)/(cnn_correct + lstm_correct)
print(f"McNemar's test intra: {mc}")

23 13 2 10
McNemar's test cross: 6.666666666666667
7 1 0 0
McNemar's test intra: 0.0
